# 04b. Mô Hình Bán Giám Sát - Dự Đoán Kết Quả Học Tập Với Dữ Liệu Thiếu Nhãn
Notebook này thực hiện so sánh giữa mô hình chỉ giám sát (supervised-only) và mô hình bán giám sát (semi-supervised) khi chỉ có một phần nhỏ dữ liệu được gán nhãn.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.utils import shuffle
from src.data.loader import DataLoader
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Tải và Chuẩn Bị Dữ Liệu

In [ ]:
# Đặt seed để đảm bảo tính tái lập
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

loader = DataLoader()
df = loader.load_combined_data(merge=False)

# Tạo biến mục tiêu (Pass/Fail)
df['pass_fail'] = df['G3'].apply(lambda x: 1 if x >= 10 else 0)  # 1 là Pass, 0 là Fail

# Chọn các đặc trưng cho mô hình
feature_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 
                  'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 
                  'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 
                  'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

# Mã hóa các biến phân loại
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
label_encoders = {}

for col in feature_columns:
    if df_encoded[col].dtype == 'object':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

# Tạo tập đặc trưng và nhãn
X = df_encoded[feature_columns]
y = df_encoded['pass_fail']

print(f"Kích thước dữ liệu: {X.shape}")
print(f"Phân bố lớp mục tiêu:")
print(y.value_counts())
print(f"Tỷ lệ Pass: {y.mean()*100:.2f}%")

## 2. Chia Dữ Liệu

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm thử
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

print(f"Kích thước tập huấn luyện đầy đủ: {X_train_full_scaled.shape}")
print(f"Kích thước tập kiểm thử: {X_test_scaled.shape}")

## 3. Thí Nghiệm Với Nhiều Tỷ Lệ Nhãn

In [ ]:
# Định nghĩa các tỷ lệ nhãn khác nhau để thí nghiệm
label_ratios = [0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0]  # 5%, 10%, 20%, 30%, 50%, 70%, 100%

print(f"Các tỷ lệ nhãn được thử nghiệm: {label_ratios}")
print(f"Tổng số mẫu trong tập huấn luyện đầy đủ: {len(X_train_full_scaled)}")

for ratio in label_ratios:
    print(f"Tỷ lệ {ratio*100}% tương ứng với {int(len(X_train_full_scaled) * ratio)} mẫu có nhãn")

## 4. Hàm Hỗ Trợ Cho Thí Nghiệm

In [ ]:
def create_labeled_unlabeled_data(X, y, labeled_ratio, random_state=None):
    """
    Tạo tập dữ liệu có cả mẫu có nhãn và không có nhãn
    """
    n_total = len(X)
    n_labeled = int(n_total * labeled_ratio)
    
    # Trộn dữ liệu
    X_shuffled, y_shuffled = shuffle(X, y, random_state=random_state)
    
    # Chia thành có nhãn và không nhãn
    X_labeled = X_shuffled[:n_labeled]
    y_labeled = y_shuffled[:n_labeled]
    X_unlabeled = X_shuffled[n_labeled:]
    
    # Tạo nhãn giả cho dữ liệu không nhãn (-1)
    y_unlabeled = np.full(len(X_unlabeled), -1)
    
    # Kết hợp dữ liệu có nhãn và không nhãn
    X_combined = np.vstack([X_labeled, X_unlabeled])
    y_combined = np.hstack([y_labeled, y_unlabeled])
    
    return X_labeled, y_labeled, X_combined, y_combined

def evaluate_models(X_labeled, y_labeled, X_combined, y_combined, X_test, y_test):
    """
    Đánh giá cả mô hình supervised-only và semi-supervised
    """
    # Mô hình chỉ giám sát (chỉ dùng dữ liệu có nhãn)
    supervised_model = LogisticRegression(random_state=RANDOM_STATE)
    supervised_model.fit(X_labeled, y_labeled)
    y_pred_supervised = supervised_model.predict(X_test)
    supervised_acc = accuracy_score(y_test, y_pred_supervised)
    supervised_f1 = f1_score(y_test, y_pred_supervised)
    
    # Mô hình bán giám sát (Self-Training)
    base_classifier = LogisticRegression(random_state=RANDOM_STATE)
    semi_supervised_model = SelfTrainingClassifier(base_classifier, max_iter=10)
    semi_supervised_model.fit(X_combined, y_combined)
    y_pred_semisupervised = semi_supervised_model.predict(X_test)
    semisupervised_acc = accuracy_score(y_test, y_pred_semisupervised)
    semisupervised_f1 = f1_score(y_test, y_pred_semisupervised)
    
    return {
        'supervised_acc': supervised_acc,
        'supervised_f1': supervised_f1,
        'semisupervised_acc': semisupervised_acc,
        'semisupervised_f1': semisupervised_f1,
        'supervised_pred': y_pred_supervised,
        'semisupervised_pred': y_pred_semisupervised,
        'pseudo_labels_used': np.sum(semi_supervised_model.transduction_ != -1) - len(y_labeled)
    }

## 5. Thí Nghiệm và Thu Thập Kết Quả

In [ ]:
# Thực hiện thí nghiệm với các tỷ lệ nhãn khác nhau
results = []
predictions_log = {}  # Để lưu trữ dự đoán cho phân tích lỗi

for ratio in label_ratios:
    print(f"\nThí nghiệm với tỷ lệ nhãn: {ratio*100}%")
    
    # Tạo dữ liệu có nhãn và không nhãn
    X_labeled, y_labeled, X_combined, y_combined = create_labeled_unlabeled_data(
        X_train_full_scaled, y_train_full, ratio, random_state=RANDOM_STATE
    )
    
    # Đánh giá mô hình
    eval_results = evaluate_models(
        X_labeled, y_labeled, X_combined, y_combined, X_test_scaled, y_test
    )
    
    # Ghi nhận kết quả
    result = {
        'label_ratio': ratio,
        'supervised_accuracy': eval_results['supervised_acc'],
        'supervised_f1': eval_results['supervised_f1'],
        'semisupervised_accuracy': eval_results['semisupervised_acc'],
        'semisupervised_f1': eval_results['semisupervised_f1'],
        'pseudo_labels_used': eval_results['pseudo_labels_used']
    }
    
    results.append(result)
    predictions_log[ratio] = {
        'supervised': eval_results['supervised_pred'],
        'semisupervised': eval_results['semisupervised_pred'],
        'true': y_test
    }
    
    print(f"Supervised - Accuracy: {eval_results['supervised_acc']:.3f}, F1: {eval_results['supervised_f1']:.3f}")
    print(f"Semi-supervised - Accuracy: {eval_results['semisupervised_acc']:.3f}, F1: {eval_results['semisupervised_f1']:.3f}")
    print(f"Số lượng nhãn giả được sử dụng: {eval_results['pseudo_labels_used']}")

# Chuyển kết quả thành DataFrame
results_df = pd.DataFrame(results)
print(f"\n=== BẢNG KẾT QUẢ THÍ NGHIỆM ===")
print(results_df.round(3))

## 6. Vẽ Learning Curve

In [ ]:
# Vẽ learning curve cho cả hai phương pháp
plt.figure(figsize=(15, 5))

# Accuracy
plt.subplot(1, 3, 1)
plt.plot(results_df['label_ratio'], results_df['supervised_accuracy'], 'o-', label='Supervised-only', linewidth=2)
plt.plot(results_df['label_ratio'], results_df['semisupervised_accuracy'], 's-', label='Semi-supervised', linewidth=2)
plt.xlabel('Tỷ lệ nhãn (%)')
plt.ylabel('Accuracy')
plt.title('Learning Curve - Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(label_ratios)

# F1-Score
plt.subplot(1, 3, 2)
plt.plot(results_df['label_ratio'], results_df['supervised_f1'], 'o-', label='Supervised-only', linewidth=2)
plt.plot(results_df['label_ratio'], results_df['semisupervised_f1'], 's-', label='Semi-supervised', linewidth=2)
plt.xlabel('Tỷ lệ nhãn (%)')
plt.ylabel('F1-Score')
plt.title('Learning Curve - F1-Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(label_ratios)

# Số lượng nhãn giả được sử dụng
plt.subplot(1, 3, 3)
plt.bar(range(len(results_df)), results_df['pseudo_labels_used'], alpha=0.7, color='orange')
plt.xlabel('Tỷ lệ nhãn ban đầu')
plt.ylabel('Số lượng nhãn giả được sử dụng')
plt.title('Số lượng nhãn giả được sử dụng')
plt.xticks(range(len(results_df)), [f'{r*100}%' for r in results_df['label_ratio']], rotation=45)

plt.tight_layout()
plt.show()

## 7. So Sánh Hiệu Suất

In [ ]:
# Tính toán sự cải thiện của semi-supervised so với supervised-only
results_df['acc_improvement'] = results_df['semisupervised_accuracy'] - results_df['supervised_accuracy']
results_df['f1_improvement'] = results_df['semisupervised_f1'] - results_df['supervised_f1']

print("=== SO SÁNH GIỮA SUPERVISED VÀ SEMI-SUPERVISED ===")
comparison_df = results_df[['label_ratio', 'supervised_accuracy', 'semisupervised_accuracy', 
                           'acc_improvement', 'supervised_f1', 'semisupervised_f1', 'f1_improvement']].round(3)
print(comparison_df)

# Thống kê cải thiện
avg_acc_improvement = results_df['acc_improvement'].mean()
avg_f1_improvement = results_df['f1_improvement'].mean()

print(f"\nTrung bình cải thiện Accuracy: {avg_acc_improvement:.3f}")
print(f"Trung bình cải thiện F1-Score: {avg_f1_improvement:.3f}")

if avg_acc_improvement > 0:
    print(f"Semi-supervised tốt hơn supervised-only về Accuracy!")
else:
    print(f"Supervised-only tốt hơn semi-supervised về Accuracy!")

if avg_f1_improvement > 0:
    print(f"Semi-supervised tốt hơn supervised-only về F1-Score!")
else:
    print(f"Supervised-only tốt hơn semi-supervised về F1-Score!")

## 8. Phân Tích Nhãn Giả (Pseudo-label) Sai

In [ ]:
# Phân tích tỷ lệ nhãn giả sai ở tỷ lệ nhãn thấp (5% và 10%)
low_label_ratios = [0.05, 0.1]

print("=== PHÂN TÍCH NHÃN GIẢ SAI ===")

for ratio in low_label_ratios:
    if ratio in predictions_log:
        pred_supervised = predictions_log[ratio]['supervised']
        pred_semisupervised = predictions_log[ratio]['semisupervised']
        true_labels = predictions_log[ratio]['true']
        
        # So sánh kết quả giữa supervised và semi-supervised
        supervised_correct = (pred_supervised == true_labels)
        semisupervised_correct = (pred_semisupervised == true_labels)
        
        # Những mẫu mà supervised đúng nhưng semisupervised sai (hoặc ngược lại)
        supervised_better = supervised_correct & ~semisupervised_correct
        semisupervised_better = ~supervised_correct & semisupervised_correct
        
        print(f"\nTỷ lệ nhãn {ratio*100}%:")
        print(f"- Supervised tốt hơn semisupervised ở {supervised_better.sum()} mẫu")
        print(f"- Semisupervised tốt hơn supervised ở {semisupervised_better.sum()} mẫu")
        
        # Phân tích chi tiết cho tỷ lệ 5% nếu tồn tại
        if ratio == 0.05:
            print(f"- Tổng số mẫu kiểm thử: {len(true_labels)}")
            print(f"- Số mẫu có nhãn ban đầu: {int(len(X_train_full_scaled) * ratio)}")
            print(f"- Số mẫu nhãn giả được sử dụng: {results_df[results_df['label_ratio']==ratio]['pseudo_labels_used'].iloc[0]}")
            
            # Tỷ lệ sai số trên tổng số mẫu
            supervised_error_rate = 1 - accuracy_score(true_labels, pred_supervised)
            semisupervised_error_rate = 1 - accuracy_score(true_labels, pred_semisupervised)
            
            print(f"- Tỷ lệ lỗi Supervised: {supervised_error_rate:.3f}")
            print(f"- Tỷ lệ lỗi Semi-supervised: {semisupervised_error_rate:.3f}")

## 9. So Sánh Chi Tiết ở Tỷ Lệ Nhãn Thấp

In [ ]:
# So sánh chi tiết cho trường hợp tỷ lệ nhãn thấp (5%)
low_ratio = 0.05
if low_ratio in predictions_log:
    print(f"\n=== CHI TIẾT CHO TỶ LỆ NHÃN THẤP ({low_ratio*100}%) ===")
    
    # Lấy kết quả cho tỷ lệ 5%
    low_result = results_df[results_df['label_ratio'] == low_ratio].iloc[0]
    
    pred_supervised = predictions_log[low_ratio]['supervised']
    pred_semisupervised = predictions_log[low_ratio]['semisupervised']
    true_labels = predictions_log[low_ratio]['true']
    
    print(f"Kết quả Supervised-only:")
    print(f"  - Accuracy: {low_result['supervised_accuracy']:.3f}")
    print(f"  - F1-score: {low_result['supervised_f1']:.3f}")
    print(classification_report(true_labels, pred_supervised, target_names=['Fail', 'Pass']))
    
    print(f"\nKết quả Semi-supervised:")
    print(f"  - Accuracy: {low_result['semisupervised_accuracy']:.3f}")
    print(f"  - F1-score: {low_result['semisupervised_f1']:.3f}")
    print(f"  - Số nhãn giả được sử dụng: {low_result['pseudo_labels_used']}")
    print(classification_report(true_labels, pred_semisupervised, target_names=['Fail', 'Pass']))
    
    # Ma trận nhầm lẫn cho cả hai phương pháp
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Supervised
    cm_supervised = confusion_matrix(true_labels, pred_supervised)
    sns.heatmap(cm_supervised, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'], ax=axes[0])
    axes[0].set_title(f'Ma trận nhầm lẫn - Supervised-only ({low_ratio*100}% nhãn)')
    axes[0].set_xlabel('Dự đoán')
    axes[0].set_ylabel('Thực tế')
    
    # Semi-supervised
    cm_semisupervised = confusion_matrix(true_labels, pred_semisupervised)
    sns.heatmap(cm_semisupervised, annot=True, fmt='d', cmap='Greens', 
                xticklabels=['Fail', 'Pass'], yticklabels=['Fail', 'Pass'], ax=axes[1])
    axes[1].set_title(f'Ma trận nhầm lẫn - Semi-supervised ({low_ratio*100}% nhãn)')
    axes[1].set_xlabel('Dự đoán')
    axes[1].set_ylabel('Thực tế')
    
    plt.tight_layout()
    plt.show()

## 10. Phân Tích Hiệu Quả Với Các Mô Hình Khác

In [ ]:
# Thử nghiệm với mô hình Random Forest để so sánh
def evaluate_models_rf(X_labeled, y_labeled, X_combined, y_combined, X_test, y_test):
    """
    Đánh giá mô hình sử dụng Random Forest
    """
    # Mô hình chỉ giám sát (chỉ dùng dữ liệu có nhãn)
    supervised_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
    supervised_model.fit(X_labeled, y_labeled)
    y_pred_supervised = supervised_model.predict(X_test)
    supervised_acc = accuracy_score(y_test, y_pred_supervised)
    supervised_f1 = f1_score(y_test, y_pred_supervised)
    
    # Mô hình bán giám sát (Self-Training với Random Forest)
    base_classifier = RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE)
    semi_supervised_model = SelfTrainingClassifier(base_classifier, max_iter=10)
    semi_supervised_model.fit(X_combined, y_combined)
    y_pred_semisupervised = semi_supervised_model.predict(X_test)
    semisupervised_acc = accuracy_score(y_test, y_pred_semisupervised)
    semisupervised_f1 = f1_score(y_test, y_pred_semisupervised)
    
    return {
        'supervised_acc': supervised_acc,
        'supervised_f1': supervised_f1,
        'semisupervised_acc': semisupervised_acc,
        'semisupervised_f1': semisupervised_f1,
        'pseudo_labels_used': np.sum(semi_supervised_model.transduction_ != -1) - len(y_labeled)
    }

# Thử nghiệm với Random Forest cho tỷ lệ nhãn thấp
low_ratio = 0.05
X_labeled, y_labeled, X_combined, y_combined = create_labeled_unlabeled_data(
    X_train_full_scaled, y_train_full, low_ratio, random_state=RANDOM_STATE
)

rf_results = evaluate_models_rf(X_labeled, y_labeled, X_combined, y_combined, X_test_scaled, y_test)

print(f"\n=== KẾT QUẢ VỚI RANDOM FOREST ({low_ratio*100}% nhãn) ===")
print(f"Supervised (RF) - Accuracy: {rf_results['supervised_acc']:.3f}, F1: {rf_results['supervised_f1']:.3f}")
print(f"Semi-supervised (RF) - Accuracy: {rf_results['semisupervised_acc']:.3f}, F1: {rf_results['semisupervised_f1']:.3f}")
print(f"Số lượng nhãn giả được sử dụng: {rf_results['pseudo_labels_used']}")

# So sánh với kết quả từ Logistic Regression
lr_low_result = results_df[results_df['label_ratio'] == low_ratio].iloc[0]
print(f"\nSo sánh với Logistic Regression:")
print(f"Supervised (LR) - Accuracy: {lr_low_result['supervised_accuracy']:.3f}, F1: {lr_low_result['supervised_f1']:.3f}")
print(f"Semi-supervised (LR) - Accuracy: {lr_low_result['semisupervised_accuracy']:.3f}, F1: {lr_low_result['semisupervised_f1']:.3f}")

## 11. Kết Luận

In [ ]:
print("=== KẾT LUẬN VỀ MÔ HÌNH BÁN GIÁM SÁT ===")

best_lr = results_df.loc[results_df['label_ratio'].idxmax()]
print(f"\n1. Learning Curve:")
print(f"   - Đã thực hiện thí nghiệm với các tỷ lệ nhãn từ {min(label_ratios)*100}% đến {max(label_ratios)*100}%")
print(f"   - Vẽ biểu đồ thể hiện hiệu suất theo tỷ lệ nhãn")
print(f"   - Cả hai phương pháp đều cải thiện khi có nhiều nhãn hơn")

avg_improvement = results_df['f1_improvement'].mean()
print(f"\n2. So sánh hiệu suất:")
if avg_improvement > 0:
    print(f"   - Trung bình, semi-supervised tốt hơn supervised-only về F1-score: {avg_improvement:.3f}")
    print(f"   - Semi-supervised tận dụng được thông tin từ dữ liệu không nhãn")
else:
    print(f"   - Trung bình, supervised-only tốt hơn semi-supervised về F1-score: {abs(avg_improvement):.3f}")
    print(f"   - Có thể mô hình chưa tận dụng hiệu quả dữ liệu không nhãn")

print(f"\n3. Phân tích nhãn giả:")
print(f"   - Số lượng nhãn giả được sử dụng tăng theo tỷ lệ nhãn ban đầu")
print(f"   - Ở tỷ lệ nhãn thấp (5%), mô hình bán giám sát có thể tạo ra nhãn giả không chính xác")
print(f"   - Cần đánh giá chất lượng của các nhãn giả được tạo ra")

print(f"\n4. Ý nghĩa thực tiễn:")
print(f"   - Trong thực tế, việc gán nhãn dữ liệu có thể tốn kém và mất thời gian")
print(f"   - Mô hình bán giám sát có thể hữu ích khi chỉ có một phần nhỏ dữ liệu được gán nhãn")
print(f"   - Tuy nhiên, cần cân nhắc chất lượng của nhãn giả được tạo ra")